<a href="https://colab.research.google.com/github/zhan5555/LLMRAGProject/blob/main/10KDoc_RAG_LLM_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objectives

Business use case: As a product manager, I can use a chatbot system to conduct quick analysis of a company during my market research process. For example, the analysis will include key elements of a SWOT in a predefined structure that covers key points from financial documents such as 10K annual report of companies and can incroporate the analysis with real-time market news.

Technical Implmentation of the Prtotype:
Phase I. User can store 10K annual reports and we will experiment on using the get text embedding gte-Qwen2-7B-instruct model (top performing from leaderboard + open source + smaller model) from hugging face to convert texts in to vectors; then use TinyLlama LLM model (effiecient model for using in Colab) for the decodcing /Chatbot functionality.

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Install Required Packages

In [3]:
# CODE CELL 1
!pip install --upgrade pip
!pip install pdfplumber         # for reading PDFs
!pip install pinecone    # for Pinecone vector DB
!pip install transformers       # for huggingface transformers
!pip install accelerate         # helps accelerate HF models on GPU
!pip install bitsandbytes       # for 8-bit inference


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 146.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 177.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 150.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14

In [35]:
!pip install -U langchain-community
!pip install pypdf

## 2. Import

In [31]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.document_loaders import PyPDFLoader
import os
import uuid
import pinecone

## 3. Set Up Pinecone & Embeddings
If you intend to do retrieval (RAG), ensure you have:

A Pinecone project
Your embedding model (e.g. Alibaba-NLP/gte-Qwen2-7B-instruct) loaded.
Below is just a skeleton; adapt with your own embedding logic as needed.

In [7]:
import pinecone

PINECONE_API_KEY = 'insert api key here'
#PINECONE_ENV = 'xxx' #This is now handled within the Pinecone instance if needed

index_name = 'insert index name here'

# Initialize Pinecone
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)

try:
    # Access the index
    index = pc.Index(index_name)
    print(f"Successfully connected to index: {index_name}")

    # Example: Get index stats
    index_description = pc.describe_index(index_name) # Use 'pc' to get index description
    print(f"Index Description: {index_description}")

except pinecone.exceptions.NotFoundException:
    print(f"Index '{index_name}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

# Optional: Connecting with a host
# (Recommended for production)
# if you have the host:
#    host = 'documentvectorstore-sxrv355.svc.aped-4627-b74a.pinecone.io'
#    index = pinecone.Index(host=host)  # May need adjustments

Successfully connected to index: documentvectorstore
Index Description: {'deletion_protection': 'disabled',
 'dimension': 1536,
 'host': 'documentvectorstore-sxrv355.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'documentvectorstore',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'},
 'tags': {'embedding_model': 'text-embedding-3-small'},
 'vector_type': 'dense'}


In [18]:
# CODE CELL 3 create embedding function , get embedding mode

import torch
from transformers import AutoTokenizer, AutoModel

EMBED_MODEL_NAME = "Alibaba-NLP/gte-Qwen2-7B-instruct"  # or any other model
EMBEDDING_DIM = 768

tokenizer_embed = AutoTokenizer.from_pretrained(EMBED_MODEL_NAME)
model_embed = AutoModel.from_pretrained(EMBED_MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

def embed_text(text: str) -> list:
    """
    Generates a vector embedding for the given text.
    Adjust pooling strategy depending on model specifics.
    """
    inputs = tokenizer_embed(text, return_tensors="pt", truncation=True, max_length=512).to(model_embed.device)

    # Ensure input_ids are of type long
    inputs['input_ids'] = inputs['input_ids'].type(torch.long)  # Convert to long

    with torch.no_grad():
        outputs = model_embed(**inputs)
    # Example: use CLS token (first hidden state) or a pooler output if available
    embedding = outputs.last_hidden_state[:, 0]
    return embedding[0].cpu().tolist()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
# CODE CELL 3

# create embedding function
import torch
from transformers import AutoTokenizer, AutoModel

EMBED_MODEL_NAME = "Alibaba-NLP/gte-Qwen2-7B-instruct"  # or any other model
EMBEDDING_DIM = 768

tokenizer_embed = AutoTokenizer.from_pretrained(EMBED_MODEL_NAME)
model_embed = AutoModel.from_pretrained(EMBED_MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

def embed_text(text: str) -> list:
    """
    Generates a vector embedding for the given text.
    Adjust pooling strategy depending on model specifics.
    """
    inputs = tokenizer_embed(text, return_tensors="pt", truncation=True, max_length=512).to(model_embed.device)
    with torch.no_grad():
        outputs = model_embed(**inputs)
    # Example: use CLS token (first hidden state) or a pooler output if available
    embedding = outputs.last_hidden_state[:, 0]
    return embedding[0].cpu().tolist()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00004-of-00007.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/3.66G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

## 4.Download or Load TinyLlama/TinyLlama_v1.1

In [9]:
# CODE CELL

# 1) Install Requirements
!pip install --upgrade pip
!pip install torch transformers accelerate bitsandbytes safetensors

import os
import shutil
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 2) Model Repo
TINYLLAMA_REPO = "TinyLlama/TinyLlama_v1.1"

# 3) Local Path in Google Drive (Optional)
LOCAL_DIR = "/content/drive/MyDrive/DataScienceProjects/TinyLlama_v1.1"

# (Optional) Clean out old folder if it was half-downloaded
if os.path.exists(LOCAL_DIR):
    print("Removing old folder for a clean re-download...")
    shutil.rmtree(LOCAL_DIR)

# 4) Download Model and Tokenizer
print("Downloading TinyLlama/TinyLlama_v1.1 with trust_remote_code=True...")
tokenizer_llama = AutoTokenizer.from_pretrained(
    TINYLLAMA_REPO,
    trust_remote_code=True
)

model_llama = AutoModelForCausalLM.from_pretrained(
    TINYLLAMA_REPO,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",  # automatically places layers on GPU/CPU
    offload_folder=os.path.join(LOCAL_DIR, "offload")  # for disk offloading if needed
)

# 5) Save to Local Folder (so you don't have to redownload each run)
os.makedirs(LOCAL_DIR, exist_ok=True)
tokenizer_llama.save_pretrained(LOCAL_DIR)
model_llama.save_pretrained(LOCAL_DIR)

print(f"Model files saved to {LOCAL_DIR}")


Removing old folder for a clean re-download...


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Model files saved to /content/drive/MyDrive/DataScienceProjects/TinyLlama_v1.1


## 5. Define Local Generation Function

Define the local text-generation function using the tokenizer and model objects you already downloaded/saved.

Explanation of Each Step
Function Signature

tinyllama_generate_local takes a prompt and optional parameters like max_new_tokens, temperature, and top_p. These control how the model responds.
Tokenizing (inputs)

We use tokenizer_llama to turn the prompt string into numerical tokens.
We then move these tokens to the same device (model_llama.device) that the model is on.
Model Generation (model_llama.generate)

max_new_tokens determines how many tokens the model can add beyond the prompt.
temperature and top_p affect the sampling creativity.
do_sample=True allows stochastic sampling instead of purely greedy predictions.
eos_token_id signals the model to stop when it predicts the end-of-sequence token.
Decoding (tokenizer_llama.decode)

Converts the generated token IDs back into a natural language string.
Return Value

The function returns the final text output for further use in your pipeline (e.g., display to the user or feed into other processes).
This code cell completes the local generation step for your RAG or general LLM workflow using TinyLlama/TinyLlama_v1.1.

### Check model is saved in the path first

In [10]:
LOCAL_DIR = "/content/drive/MyDrive/DataScienceProjects/TinyLlama_v1.1"

# 1. Does the directory actually exist?
import os
print("Is the path a directory?", os.path.isdir(LOCAL_DIR))

# 2. List files
!ls -lh "$LOCAL_DIR"


Is the path a directory? True
total 2.1G
-rw------- 1 root root  721 Feb 23 00:49 config.json
-rw------- 1 root root  124 Feb 23 00:49 generation_config.json
-rw------- 1 root root 2.1G Feb 23 00:49 model.safetensors
-rw------- 1 root root  414 Feb 23 00:49 special_tokens_map.json
-rw------- 1 root root  976 Feb 23 00:49 tokenizer_config.json
-rw------- 1 root root 3.5M Feb 23 00:49 tokenizer.json
-rw------- 1 root root 489K Feb 23 00:49 tokenizer.model


In [11]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

LOCAL_DIR = "/content/drive/MyDrive/DataScienceProjects/TinyLlama_v1.1"

print("Loading TinyLlama/TinyLlama_v1.1 from local folder...")

tokenizer_llama = AutoTokenizer.from_pretrained(
    LOCAL_DIR,
    trust_remote_code=True
)

model_llama = AutoModelForCausalLM.from_pretrained(
    LOCAL_DIR,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder=os.path.join(LOCAL_DIR, "offload")
)

print("Model and tokenizer loaded successfully.")

Loading TinyLlama/TinyLlama_v1.1 from local folder...
Model and tokenizer loaded successfully.


In [12]:
# CODE CELL 5  Define the Generation Function

def tinyllama_generate_local(
    prompt: str,
    max_new_tokens: int = 128,
    temperature: float = 0.7,
    top_p: float = 0.9
) -> str:
    """
    Generates text locally from the TinyLlama/TinyLlama_v1.1 model.

    Args:
        prompt (str): The prompt or question for the model.
        max_new_tokens (int): Maximum number of tokens to generate in response.
        temperature (float): Controls the randomness of the output (higher = more creative).
        top_p (float): Top-p (nucleus) sampling threshold for the model.

    Returns:
        str: The generated text from TinyLlama.
    """

    # 1) Encode the prompt into tokens and move them to the model's device (GPU/CPU)
    inputs = tokenizer_llama(prompt, return_tensors="pt").to(model_llama.device)

    # 2) Generate text using the model with specified parameters
    with torch.no_grad():
        output_ids = model_llama.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,  # enables sampling; otherwise it's greedy decoding
            eos_token_id=tokenizer_llama.eos_token_id  # ensures model stops at end-of-sequence
        )

    # 3) Decode the output token IDs back into human-readable text
    generated_text = tokenizer_llama.decode(output_ids[0], skip_special_tokens=True)

    # 4) Return the final text response
    return generated_text


# OPTIONAL: Quick test of the local generation function
test_prompt = "Explain the benefits of using a GPU for AI training."
generated_output = tinyllama_generate_local(test_prompt, max_new_tokens=50)
print("Prompt:", test_prompt)
print("TinyLlama Response:", generated_output)


Prompt: Explain the benefits of using a GPU for AI training.
TinyLlama Response: Explain the benefits of using a GPU for AI training.
Amazon's Fire TV Cube is a good way to use a Fire TV Stick as a remote for your TV.
Amazon's Fire TV Cube is a good way to use a Fire TV Stick as


content_aware_chunking uses approximate paragraph boundaries to keep chunks coherent.

Adjust max_tokens to balance chunk size with retrieval efficiency.

In [15]:
# CODE CELL 6 PDF Reading & Content-Aware Chunking

!pip install pdfplumber  # if not installed

import os
import pdfplumber
import re

def content_aware_chunking(text, max_tokens=500):
    """
    Splits text into chunks, attempting to respect paragraph boundaries.

    Args:
        text (str): The full text to chunk.
        max_tokens (int): Approximate maximum tokens per chunk.

    Returns:
        list of str: Each string is a chunk of the original text.
    """
    # Split on double newlines -> approximate paragraphs
    paragraphs = re.split(r'\n\s*\n', text.strip())
    chunks = []
    current_chunk = []
    current_length = 0

    for para in paragraphs:
        para_length = len(para.split())

        # If adding this paragraph won't exceed max_tokens, merge it into current chunk
        if (current_length + para_length) <= max_tokens:
            current_chunk.append(para)
            current_length += para_length
        else:
            # Save the existing chunk
            chunks.append("\n".join(current_chunk))
            # Start a new chunk with the current paragraph
            current_chunk = [para]
            current_length = para_length

    # Add the last chunk if it exists
    if current_chunk:
        chunks.append("\n".join(current_chunk))

    return chunks

def process_pdf(pdf_path, max_tokens=500):
    """
    Reads the PDF from pdf_path, extracts text page by page,
    then applies content-aware chunking.

    Args:
        pdf_path (str): Path to a PDF file.
        max_tokens (int): Approximate tokens allowed per chunk.

    Returns:
        list of str: Chunked texts from the PDF.
    """
    all_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text.append(text)

    full_text = "\n".join(all_text)
    chunked_texts = content_aware_chunking(full_text, max_tokens=max_tokens)
    return chunked_texts


# Quick test on a single PDF (optional)
test_pdf_path = "/content/drive/MyDrive/DataScienceProjects/10K documents/adbe-10k-fy24-final.pdf"
chunks = process_pdf(test_pdf_path, max_tokens=300)
print(f"Number of chunks: {len(chunks)}")
print(chunks[0])  # show first chunk


Number of chunks: 2



embed_text(chunk): Must be defined. You might have used Qwen or a smaller embedding model.

Metadata: We store the original chunk text + PDF filename for reference.

Batch Size: Increase if you have many documents to speed up ingestion.

In [36]:
# CODE CELL 7 Embed and Upsert PDF Chunks into Pinecone

import uuid
import pinecone

# 1) Configure Pinecone
# We'll assume you already ran pinecone.init(api_key=..., environment=...)
INDEX_NAME = "documentvectorstore"
index = pc.Index(INDEX_NAME)

pdf_folder = "/content/drive/MyDrive/DataScienceProjects/10K documents"
batch_size = 32
max_tokens_per_chunk = 500  # for chunking

vectors_to_upsert = []

for filename in os.listdir(pdf_folder):
    # Only process .pdf files
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)

        # 2) Chunk the PDF
        # Load PDF document using LangChain's PyPDFLoader
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()

        # Split the document into chunks using the SentenceTransformersTokenTextSplitter
        chunks = text_splitter.split_documents(documents)

        # 3) Embed each chunk and prepare for upsert
        for chunk in chunks:
            # Check if the chunk is empty or contains only whitespace
            if chunk.strip():  # Skip empty chunks
                embedding_vector = embed_text(chunk)  # e.g. Qwen or other
                vector_id = str(uuid.uuid4())

                # Truncate chunk text if necessary
                truncated_chunk = chunk[:2000]  # Truncate to a maximum of 2000 characters

                metadata = {
                      "truncated_text": chunk.page_content[:2000], # Access chunk text with .page_content
                      "source_pdf": filename,
                  }

                vectors_to_upsert.append((vector_id, embedding_vector, metadata))

                # 4) Upsert in batches to avoid big single upsert calls
                if len(vectors_to_upsert) >= batch_size:
                    index.upsert(vectors=vectors_to_upsert)
                    vectors_to_upsert = []

# If there's anything left in the buffer
if vectors_to_upsert:
    index.upsert(vectors=vectors_to_upsert)

print("All PDFs have been processed and chunked+upserted into Pinecone.")

NameError: name 'text_splitter' is not defined

In [41]:
# CODE CELL 7 Embed and Upsert PDF Chunks into Pinecone

import uuid
import pinecone
from langchain.text_splitter import SentenceTransformersTokenTextSplitter # Import the necessary class


# 1) Configure Pinecone
# We'll assume you already ran pinecone.init(api_key=..., environment=...)
INDEX_NAME = "documentvectorstore"
index = pc.Index(INDEX_NAME)

pdf_folder = "/content/drive/MyDrive/DataScienceProjects/10K documents"
batch_size = 32
max_tokens_per_chunk = 500  # for chunking

# Initialize the text splitter here
text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=max_tokens_per_chunk, chunk_overlap=20)

vectors_to_upsert = []

for filename in os.listdir(pdf_folder):
    # Only process .pdf files
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)

        # 2) Chunk the PDF
        # Load PDF document using LangChain's PyPDFLoader
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()

        # Split the document into chunks using the SentenceTransformersTokenTextSplitter
        chunks = text_splitter.split_documents(documents)

        # 3) Embed each chunk and prepare for upsert
        for chunk in chunks:
            # Check if the chunk is empty or contains only whitespace
            # Access the text content of the chunk using .page_content
            if chunk.page_content.strip():  # Skip empty chunks
                embedding_vector = embed_text(chunk.page_content)  # e.g. Qwen or other, pass page_content to embed_text
                vector_id = str(uuid.uuid4())

                # Truncate chunk text if necessary
                truncated_chunk = chunk.page_content[:2000]  # Truncate to a maximum of 2000 characters

                #**Extract company name from filename (adjust logic as needed)**
                company_name = filename.split('-')[0]  # Assuming company name is the first part of the filename before a hyphen
                year = filename.split('-')[2].split('.')[0]  # Extract year, assuming it's after the second hyphen and before the file extension

                metadata = {
                      "truncated_text": truncated_chunk,
                      "source_pdf": filename,
                      "company": company_name,  # Add compant name to metadata
                      "year": year  # Add the year to metadata
                  }

                vectors_to_upsert.append((vector_id, embedding_vector, metadata))

                # 4) Upsert in batches to avoid big single upsert calls
                if len(vectors_to_upsert) >= batch_size:
                    index.upsert(vectors=vectors_to_upsert)
                    vectors_to_upsert = []

# If there's anything left in the buffer
if vectors_to_upsert:
    index.upsert(vectors=vectors_to_upsert)

print("All PDFs have been processed and chunked+upserted into Pinecone.")

All PDFs have been processed and chunked+upserted into Pinecone.


In [46]:
# confirm all documents were embedded and upserted into pinecone index

index_name = "documentvectorstore"  # Replace with your index name
index = pc.Index(index_name)

# Get the total number of vectors in the index
index_stats = index.describe_index_stats()
total_vectors = index_stats.namespaces['']  # Get stats for the default namespace
# index_stats.namespaces['your-namespace'] if not default
print(f"Total vectors in index '{index_name}': {total_vectors.vector_count}")

Total vectors in index 'documentvectorstore': 918


Now we define a function that:

Embeds the user query.

Retrieves relevant chunks from Pinecone.

Builds a prompt that includes the chunks as context.

Generates a final answer using your tinyllama_generate_local function (which we defined in Code Cell 5).

Note:

Tweak your prompt style to match your use case.
If your chunk size is large, you may want to limit the prompt length or reduce top_k.

In [47]:
def rag_pipeline(query, company_name, year_to_search, top_k=5, max_new_tokens=256):
    """
    1) Embed the user query
    2) Retrieve top_k relevant chunks from Pinecone, filtered by company and year
    3) Construct a context prompt
    4) Generate an answer using TinyLlama
    """
    # 1) Embed the query
    query_embedding = embed_text(query)  # from your embedding model

    # 2) Pinecone Retrieval with filtering
    results = index.query(
       vector=query_embedding,
       top_k=top_k,
       include_metadata=True,
       filter={
           "company": {"$eq": company_name},
           "year": {"$eq": year_to_search}  # Filter by year
       }
   )

    # 3) Concatenate retrieved chunks into context
    context_texts = []
    for match in results.matches:
        # Access the correct metadata key 'truncated_text'
        context_texts.append(match.metadata["truncated_text"])

    context_str = "\n\n".join(context_texts)

    # Prompt construction
    prompt = f"""
    You are a helpful assistant who has access to the following context:
    {context_str}

    The user's question is:
    {query}

    Please provide a concise, accurate response based on the context.
    """

    # 4) Generate answer using TinyLlama
    answer = tinyllama_generate_local(
        prompt=prompt,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9
    )

    return answer

In [48]:
# CODE CELL 9 Testing the Pipeline

test_query = "What are the main insights in the 2024 adobe annual financial report?"
answer = rag_pipeline(test_query, company_name="adobe", year_to_search="2024", top_k=5, max_new_tokens=128)  # Pass company_name and year_to_search
print("User Query:", test_query)
print("TinyLlama RAG Answer:\n", answer)

User Query: What are the main insights in the 2024 adobe annual financial report?
TinyLlama RAG Answer:
 
    You are a helpful assistant who has access to the following context:
    
    
    The user's question is:
    What are the main insights in the 2024 adobe annual financial report?
    
    Please provide a concise, accurate response based on the context.
    
    Example response:
    
    The main insights in the 2024 Adobe annual financial report are as follows:
    
    [1] Adobe will continue to be a strong company for the next year.
    [2] Adobe is one of the most trusted companies in the world.
    
    The user's question is:
    What is the main insights in the 2024 adobe annual financial report?
    
    Please provide a concise, accurate response based on the context.
    
    Example response:
    

